EEG Spike-Wave visualization tool
Copyright (C) 2023  Leonardo Cañete-Sifuentes, Azul Salmerón Ruiz, Lizbeth Becerril Belio 

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.

Repository: https://github.com/LeonardoC/SpikeWave-Visualization

In [32]:
import pandas as pd
from linelength_event_detector.lleventdetector import *
from linelength_event_detector.lltransform import *
import plotly.express as px
import plotly.graph_objects as go
from process_data import *
from scipy.signal import find_peaks, peak_prominences

In [57]:
data = pd.read_csv(r'C:\Users\L03109567\Desktop\O4-E10,112,13.csv', header = None)
data = data.to_numpy()
data = data.transpose()

In [81]:
sfx = 500 # Sampling frequency
mel = 500 # Minimum event length (milliseconds)
prc = 99
llw = 11
events = detect_events(data, 0, sfx, mel, llw, prc) 
subject_windows, subject_channels = split_events(events)

In [84]:
sfx = 500 # Sampling frequency
mel = 500 # Minimum event length (milliseconds)
prc = 99
llw = 10
actual = lltransform(data[0], sfx, llw)
events = lleventdetector([actual], sfx, prc, mel, llw)[0]
len(events)

3

In [91]:
events

array([[ 121413.,  125593.],
       [ 732598.,  739097.],
       [1131999., 1139442.]])

In [90]:
events + [1, 2]

array([[ 121414.,  125595.],
       [ 732599.,  739099.],
       [1132000., 1139444.]])

In [66]:
start = int(events[0][0])
end = int(events[0][1])
subject = int(events[0][2])

subject_data = data[subject-1][start:end+1]
nsamples = subject_data.shape[0]
time = [i / sfx for i in range(nsamples)]

peaks, _ = find_peaks(-subject_data)
mean_prominences = peak_prominences(-subject_data, peaks)[0].mean()

peaks, _ = find_peaks(-subject_data, prominence = mean_prominences)

fig = go.Figure(data = go.Scatter(
    x=time,
    y=subject_data))
fig.add_trace(go.Scatter(
    x=[time[i] for i in peaks],
    y=subject_data[peaks],
    mode='markers'))

fig.show()
